2016-10-20 일 뉴스에서 하나의 뉴스와 비슷한 뉴스를 찾아봅니다.

In [1]:
import config
from navernews_10days import get_bow, get_news_paths
from soynlp.utils import DoublespaceLineCorpus

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')
path = get_news_paths(date='2016-10-20')
docs = [doc for doc in DoublespaceLineCorpus(path, iter_sent=False)]

print(x.shape)
print(len(docs))

soynlp=0.0.49
added lovit_textmining_dataset
(30091, 9774)
30091


임의의 문서 하나를 선택하였습니다. 도널드 트럼프 vs 힐러리 클린턴의 미국 대선에 대한 뉴스입니다.

이 뉴스의 term frequency vector 는 x[4,:] 으로 선택할 수 있습니다. 

In [2]:
query_idx = 15
docs[query_idx]

'클린턴 득표율 50 목표 가능성 아주 크다  워싱턴 연합뉴스 강영두 특파원 미국 민주당 대선후보 힐러리 클린턴이 다음달 8일 대선에서 400명 이상의 선거인단을 확보하면서 대승을 거둘 수 있다는 분석이 나왔다  미국 민주당 대선후보 힐러리 클린턴이 지난 10일 현지시간 오하이오 주 콜럼버스에서 유세하는 모습 연합뉴스  미 공화당 전략가인 스티브 슈미트는 19일 현지시간 방송의 모닝조 프로그램 인터뷰에서 이 같이 전망했다  슈미트는 2008년 존 매케인의 대선캠프에 관여했다  간접선거인 미 대선에서 승리에 필요한 선거인단 매직 넘버 는 전체 538명 의 과반인 270명이다  슈미트는 현 상황은 클린턴이 선거인단 400명을 넘기는 추세 라고 분석했다  그는 대선과 함께 치러지는 연방 상 하원의원 선거에 대해서도 상원은 민주당이 장악할 것으로 보이며 하원도 민주당이 장악할 수 있는 쪽으로 근접해 가고 있다 고 관측했다  현재는 공화당이 상원과 하원을 모두 장악하고 있다  슈미트는 만약 선거가 오늘 열린다고 가정하면 공화당은 하원에서 25석을 잃게 될 것으로 보인다 면서 공화당 입장에서는 선거판의 흐름을 보여주는 추세선이 잘못된 방향으로 가고 있다 내부에서 패닉이 시작됐다 고 말했다  미 일간 워싱턴포스트 가 서베이몽키와 15개 경합주에 대한 공동 여론조사 10월8 16일 1만7천379명 를 실시해 전날 발표한 결과를 보면 클린턴은 9개 주에서 공화당의 도널드 트럼프를 앞섰고 선거인단 304명을 확보한 것으로 나타났다  공화당 주변에서조차 트럼프의 완패 가능성이 거론되는 가운데 클린턴 진영은 득표율 50 고지 점령을 새로운 목표로 내걸었다  클린턴 측은 트럼프가 음담패설 비디오 파문과 성추문 논란에 휩싸이면 사실상 지지율을 견인할 추동력을 잃은 것으로 판단하고 있다 다만 자유당 게리 존슨 등 제3 후보 의 지지율이 10 안팎으로 상대적으로 높다는 점이 변수다  클린턴 캠프의 중량감 있는 한 인사는 의회전문매체 더힐 과의 인터뷰에서 득표율 50 전망에 대해 가능성이 

앞서 연습하였던 LSHFores t를 이용하여 Cosine distance 기준으로 유사한 문서를 찾겠습니다. training은 lsh.fit(x) 로 끝납니다. 패러메터는 기본값을 이용하였습니다. 

In [3]:
%%time

from sklearn.neighbors import LSHForest

lsh = LSHForest()
lsh.fit(x)

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/sklearn/neighbors/approximate.py:258: DeprecationWarning: LSHForest has poor performance and has been deprecated in 0.19. It will be removed in version 0.21.
  DeprecationWarning)


CPU times: user 892 ms, sys: 4 ms, total: 896 ms
Wall time: 892 ms


2317번 문서와 유사한 10개의 문서를 선택하였습니다. 가장 비슷한 문서는 2317번이어야 합니다. 

In [4]:
dist, idxs = lsh.kneighbors(x[query_idx,:], n_neighbors=10)

2317번을 포함한 10개의 문서에 대하여 각 문서 별 200글자까지만 snippest를 출력하여 그 내용을 확인하겠습니다. 이와 같이 비슷한 문서를 찾는 문제에서는 LSHForest를 쓰면 편리합니다. 

또한, 문서는 어느 정도 길이가 긴 문서인 것이 좋습니다. 짧은 문서는 too much sparse하여 인덱싱이 잘 되지 않을 수 있습니다. 

In [5]:
for d, idx in zip(dist[0], idxs[0]):
    print('[idx = {}, cos = {:.3}] {} ...\n'.format(idx, 1 - d, docs[idx][:200]))

[idx = 15, cos = 1.0] 클린턴 득표율 50 목표 가능성 아주 크다  워싱턴 연합뉴스 강영두 특파원 미국 민주당 대선후보 힐러리 클린턴이 다음달 8일 대선에서 400명 이상의 선거인단을 확보하면서 대승을 거둘 수 있다는 분석이 나왔다  미국 민주당 대선후보 힐러리 클린턴이 지난 10일 현지시간 오하이오 주 콜럼버스에서 유세하는 모습 연합뉴스  미 공화당 전략가인 스티브 슈미트는 1 ...

[idx = 15006, cos = 0.846] 미국 민주당 대선후보 힐러리 클린턴이 다음달 8일 대선에서 400명 이상의 선거인단을 확보하면서 대승을 거둘 수 있다는 분석이 나왔다  미 공화당 전략가인 스티브 슈미트는 현지시간 19일 방송의 모닝조 프로그램 인터뷰에서 이 같이 전망했다 슈미트는 2008년 존 매케인의 대선캠프에 관여했다 간접선거인 미 대선에서 승리에 필요한 선거인단 매직 넘버 는 전체  ...

[idx = 11309, cos = 0.698] 공화 상 하원선거도 고전할 듯 2016년 미국 대선이 19일 앞으로 다가온 가운데 민주당 후보 힐러리 클린턴 전 국무장관이 400명 이상의 선거인단을 확보해 압승을 거둘 것이라는 전망이 제기됐다 미 공화당 전략가인 스티브 슈밋은 19일 방송의 모닝조 프로그램에 출연 클린턴이 당선 매직넘버인 선거인단 270명을 훌쩍 넘어선 400명을 확보해 공화당 도널드 트 ...

[idx = 8380, cos = 0.56] 세차례 토론을 모두 끝내고 20일 앞으로 다가온 미국 대통령 선거 11월 8일 에서 힐러리 클린턴 민주당 후보의 우세가 이어지고 있다 미국 대선에서 승리에 필요한 선거인단 과반 270명의 매직넘버 확보에서 클린턴은 공화당 도널드 트럼프 후보를 크게 앞서고 있다  19일 현지시간 에 따르면 이날 기준 미국 50개주 민주 공화당 지지 판세에서 클린턴은 매직넘버 ...

[idx = 8441, cos = 0.486] 미국 대선에 무소속으로 출마한 에번 맥멀린이 투표를 20일 앞두고 공개된 유타주 여론조사